In [12]:
! pip install pycuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import driver, gpuarray

In [13]:
from PIL import Image
import numpy as np
import math
import time

In [14]:
# ядро и вспомогательные элементы на c++
mod = SourceModule("""

 texture<int, 2, cudaReadModeElementType> image;

 const float sigma_d = 0.1;
 const float sigma_s = 12;

__device__ float gauss(float x) {
    return sigma_d * exp(-(x * x) / sigma_s);
}

__global__ void kernel(int* out, int n, int m, int kernel_size)
  {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i >= n || j >= m) {
        return;
    }

    int min_i = max(0, i - kernel_size);
    int min_j = max(0, j - kernel_size);
    int max_i = min(n-1, i + kernel_size);
    int max_j = min(m-1, j + kernel_size);

    int c_n = (max_i - min_i) + 1;
    int c_m = (max_j - min_j) + 1;

    float k = 0;
    float output = 0;
    float g, r, f;

    float center = tex2D(image, j, i) / 255.0;

    for (int c_i = 0; c_i < c_n; ++c_i) {
        for (int c_j = 0; c_j < c_m; ++c_j) {
            f = tex2D(image, min_j + c_j, min_i + c_i) / 255.0;
            g = gauss(f - center);
            r = gauss(pow(double(min_i + c_i - i), 2) + pow(double(min_j + c_j - j), 2));

            output += f * g * r;
            k += g * r;

        }
    }

    out[i * m + j] = ((int)(255 * output / k)) % 255;
  }
  """)

# функция для динамического подсчета размерностей грида и блоков
def count_cuda_dims(n, m):
    xthreadsPerBlock = n if n < 32 else 32
    ythreadsPerBlock = m if m < 32 else 32
    blocksPerGrid = (math.ceil(n / xthreadsPerBlock), math.ceil(m / ythreadsPerBlock), 1)
    threadsPerBlock = (xthreadsPerBlock, ythreadsPerBlock, 1)
    return blocksPerGrid, threadsPerBlock


# функция для применения фильтра на GPU
def filter_gpu(inp, n, m, kernel_size):
  start_time = time.time()
  kernel_size = (kernel_size - 1) / 2

  tex_image = mod.get_texref("image")
  inp = inp.astype(np.int32)
  driver.matrix_to_texref(inp, tex_image, order="C")

  out = np.zeros(n*m)
  out = out.astype(np.int32)
  out_gpu = driver.mem_alloc(out.nbytes)
  driver.memcpy_htod(out_gpu, out)

  blocksPerGrid, threadsPerBlock=count_cuda_dims(n, m)
  kernel = mod.get_function("kernel")
  kernel(out_gpu, np.int32(n), np.int32(m), np.int32(kernel_size), block=threadsPerBlock, grid = blocksPerGrid)

  driver.Context.synchronize()
  driver.memcpy_dtoh(out, out_gpu)
  return out, time.time()-start_time

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


In [15]:
sigma_d = 0.1
sigma_s = 12

def gauss(x):
    return sigma_d * math.exp(-(x * x) / sigma_s);


# функция для применения фильтра на CPU
def filter_cpu(inp, n, m, kernel_size):
    start_time = time.time()
    output = np.zeros(n*m)
    kernel = (kernel_size - 1) / 2

    for i in range(n):
        for j in range(m):
            min_i = max(0, i - kernel)
            min_j = max(0, j - kernel)
            max_i = min(n-1, i + kernel)
            max_j = min(m-1, j + kernel)

            c_n = int((max_i - min_i) + 1)
            c_m = int((max_j - min_j) + 1)

            k = 0
            value = 0
            index0 = i * m + j
            for c_i in range(c_n):
                for c_j in range(c_m):
                    index = int((min_i + c_i) * m + (min_j + c_j))
                    g = gauss(inp[index]/255.0-inp[index0]/255.0)
                    r = gauss((min_i + c_i-i)**2+(min_j + c_j-j)**2)
                    f = inp[index] / 255.0

                    value += f * r * g
                    k += g * r

            output[i * m + j] = int((255*value / k)) % 255

    return output, time.time()-start_time

In [16]:
WIDTH = 400
HIDTH = 400
img=Image.open('Mona_Lisa_GS2.bmp').convert('L')
new_img = img.resize((WIDTH,HIDTH))
new_img.save('new_image.bmp')

In [17]:
img=Image.open('Mona_Lisa_GS2.bmp').convert('L')
new_img = img.resize((WIDTH,HIDTH))
img_array = np.asarray(new_img).reshape(WIDTH*HIDTH)
res_cpu, time_cpu = filter_cpu(img_array, WIDTH, HIDTH, 5)
res_cpu = res_cpu.reshape((WIDTH, HIDTH))
PIL_image = Image.fromarray(np.uint8(res_cpu))
print(time_cpu)
PIL_image.save('CPU.bmp')
PIL_image.size

29.297870874404907


(400, 400)

In [18]:
img=Image.open('Mona_Lisa_GS2.bmp').convert('L')
new_img = img.resize((WIDTH,HIDTH))
img_array = np.array(new_img)
res_gpu, time_gpu = filter_gpu(img_array, WIDTH, HIDTH, 9)
print(time_gpu)
res_gpu = res_gpu.reshape((WIDTH, HIDTH))
PIL_image.save('GPU.bmp')
PIL_image.size

0.040471792221069336


(400, 400)

In [19]:
A = time_cpu/time_gpu
A

723.908412910675